# Conversation starter

This notebook goes over the basics of interacting with a Large Language Model using the Python library [LangChain](https://www.langchain.com/). Most of the example code here is adapted from the official [LangChain docs](https://python.langchain.com/docs/get_started/quickstart).

We will be using the OpenAI API to interact with GPT 3.5. This requires an API key and some additional steps for setup. Make sure you follow the instructions in the [README](README.md#getting-started) to get started. There is also additional information in the introductory [slide deck](../ppt/chatty-documents.pptx).

<div class="alert alert-block alert-info">

**Note:** The OpenAI API is a paid service and running this notebook will cause the owner of the API key to get billed! GPT 3.5 costs only fractions of a cent per token (roughly 3/4 of a word), but you still should be mindful not to run unnecessary prompts, especially if they contain a lot of words.

</div>


## LangChain: A framework for LLM applications

_LangChain_ does not provide any Large Language Model by itself. Instead, it is a flexible framework that provides numerous components to build LLM-powered applications. Among other things, _LangChain_ offers a uniform interface to many Large Language Models, even though the LLMs themselves must be provided from elsewhere.

For example: If we want to use GPT 3.5 in a _LangChain_ application, we need to install OpenAI's Python library `openai`, as well. When we write the actual code, we only need to interact with _LangChain_ objects, but _LangChain_ calls the `openai` library "under-the-hood". This lets us use many LLMs in _LangChain_, so we can easily swap out the LLM in our application with hardly any changes to the code.

_LangChain_'s content can be roughly divided in two categories:

- **Components**: Useful abstractions for working with LLMs that help us quickly implement common problems
- **Chains**: A collection or sequence of components performing a particular task. We can use some pre-defined chains, for common tasks, implement our own, or do a mix of both.

This notebook will show you some examples from both of these categories.


## Loading our API key

At this point you should have set up a file named `secrets.env` with your OpenAI API key. We will now use a lightweight Python package called `dotenv` to read in this file and set its contents as environment variables:


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv("../secrets.env")

os.getenv(
    "OPENAI_API_KEY"
) is not None  # Do not print the key itself! We want to keep it secret

## Accessing a Large Language Model

There are two types of language models, which in LangChain are called:

- LLMs: this is a language model which takes a single string as input and returns a single string
- ChatModels: this is a language model which takes a list of messages as input and returns a message

We use separate objects for each kind of model. Here, we will create an instance of each kind of model from the [GPT 3.5 family of models](https://platform.openai.com/docs/models/gpt-3-5):


In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI


llm = OpenAI(model="gpt-3.5-turbo-instruct")
chat_model = ChatOpenAI(model="gpt-3.5-turbo")

And that's it! _LangChain_ takes care of all everything else for us behind the scenes: Authenticating us using the API key, routing our requests to the right endpoints, retrieving and parsing the response. In the next section, we will take a look at how we can send a prompt to these models.


## Basic prompting using strings

The most straightforward way to send a prompt to the model is to use a string and `predict()` method, which is implemented by bothmodels:


In [ ]:
print(llm.predict("What is a Large Language Model?"))

In [ ]:
print(
    chat_model.predict(
        "What is the difference between a Large Language Model and a Chat Model?"
    )
)

Note that the chat model is essentially state-less and does not have any "memory" of the conversation in this case:


In [ ]:
print(chat_model.predict("Tell me more about that."))

## Prompting with `Message`s

An alternative to using a plain string to prompt a model is to use a `Message` object. Such objects contain the actual prompt, but they also contain a role descriptor in their `type` property:


In [ ]:
from langchain.schema import HumanMessage

text = "What would be a good name for my dog?"

message = HumanMessage(content=text)
message.content, message.type

You can prompt an LLM with a list of messages using the `predict_messages()` method. In that case, the model returns another `Message` object in response:


In [ ]:
response = llm.predict_messages([message])
print(response.type)
print(response.content)

In [ ]:
response = chat_model.predict_messages([message])
print(response.type)
print(response.content)

When using a chat model, you can now continue the conversation by just keeping a running list of the messages (both human and AI):


In [ ]:
conversation = [message, response]
new_prompt = HumanMessage(
    content="Can you tell me a word that rhymes with the last name in that list?"
)
conversation.append(new_prompt)
print(chat_model.predict_messages(conversation))

<div class="alert alert-block alert-info">

Keeping track of the conversation manually can become rather tedious. _LangChain_ therefore offers extensive support to streamline this process (called [Memory](https://python.langchain.com/docs/modules/memory/)), but it is beyond the scope of this session.

</div>


## Prompt templates

Large Language Model applications often use very similar prompts over and over again with only minor changes to their content.

Continuing the previous example, we might want to ask for names for all of our different pets. In that case, we would ask the same question repeatedly, only changing the kind of pet we are asking about.

For such use cases, _LangChain_ offers the `PromptTemplate` object. This object allows us to create a template with a placeholder that we can replace with whatever string we like:


In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for my {pet}?")

prompt.format(pet="lizard")

Now that we have such a template, we can iterate over a list of different kinds of pets and ask for name suggestions with very little code:


In [ ]:
pets = ["dog", "cat", "lizard", "goldfish"]
for pet in pets:
    print("*" * 10 + pet + "*" * 10)
    print(llm.predict(prompt.format(pet=pet)))
    print("\n\n")

In [ ]:
for pet in pets:
    print("*" * 10 + pet + "*" * 10)
    print(chat_model.predict(prompt.format(pet=pet)))
    print("\n\n")

A more advanced version of the prompt template is based on `Message`s. In this `ChatPromptTemplate`, we can combine our regular prompt with a special system message that instructs the model how to behave in the conversation.

<div class="alert alert-block alert-info">

There is nothing truly special about the system message per se: It is also just a string. However, it is marked as "system", as opposed to "ai" or "human", and language models can be tuned to pay particular attention to this message to adapt their output style.

</div>


In [ ]:
from langchain.prompts.chat import ChatPromptTemplate

system_template = "Respond to every prompt as if you were a {persona}."
human_template = "What is a good name for my {pet}?"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("human", human_template),
    ]
)

chat_prompt.format_messages(persona="cat", pet="dog")

Let's use this template to ask for more pet names:


In [ ]:
for pet in pets:
    print("*" * 10 + pet + "*" * 10)
    print(
        llm.predict_messages(
            chat_prompt.format_messages(persona="cat", pet=pet)
        ).content
    )
    print("\n\n")

In [ ]:
for pet in pets:
    print("*" * 10 + f" {pet} " + "*" * 10)
    print(
        chat_model.predict_messages(
            chat_prompt.format_messages(persona="cat", pet=pet)
        ).content
    )
    print("\n\n")

## Getting structured output for further processing

Passing messages back and forth makes sense for a chat-like application, but often we want the model to do some sort of analysis and return structured results. To accomplish that, we can ask the LLM to provide the output data in a certain format and then pass the response through a matching parser.

We could use any parser or parsing library we like, but _LangChain_ comes with a parser base object that plays nicely with the other components, and that we can customize to fit our needs.

<div class="alert alert-block alert-info">

If you are not familiar with the concept of _inheritance_ or _subclassing_, you can check out our workshop on [Object-Oriented Programming](https://git.dartmouth.edu/lib-digital-strategies/RDS/workshops/computational-tools/classy-code), which may be helpful to fully understand the next section.

</div>

To get the pet names as a comma-separated list, for example, we would create a coresponding parser like so:


In [ ]:
from langchain.schema import BaseOutputParser


class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")


parser = CommaSeparatedListOutputParser()
parser.parse("One, two")

And now we can pass the response from the model to the parser:


In [ ]:
system_template = """You are a helpful assistant who generates comma separated lists. You ONLY return the comma separated lists, nothing else."""
human_template = "List 5 good names for my {pet}?"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("human", human_template),
    ]
)
for pet in pets:
    print(
        parser.parse(
            chat_model.predict_messages(chat_prompt.format_messages(pet=pet)).content
        )
    )

## Putting it all together

By now, we have created several components that together make up a kind of processing pipeline:

- Use a specified prompt template
- Feed it to a specified LLM
- Parse the LLM's output with a specific parser

These steps can be combined into a so-called _chain_. To that end, _LangChain_ defines a declarative way of expressing this concatenation using the [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/expression_language).

Here is how we can define the chain above using the LCEL:


In [ ]:
chain = chat_prompt | chat_model | parser

We can then use the chain with the `invoke()` method. This method requires a dictionary that contains all the parameters we need to specify within the chain (e.g., to fill in the gaps in the prompt template) :


In [ ]:
for pet in pets:
    print(chain.invoke({"pet": pet}))

In the [next notebook](02-summarizing.ipynb), we will create a chain to do a slightly more meaningful task: Summarize text.

<table >
<tbody>
  <tr>
    <td style="padding:0px;border-width:0px;vertical-align:center">    
    Created by Simon Stone for Dartmouth College Library under <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons CC BY-NC 4.0 License</a>.<br>For questions, comments, or improvements, email <a href="mailto:researchdatahelp@groups.dartmouth.edu">Research Data Services</a>.
    </td>
    <td style="padding:0 0 0 1em;border-width:0px;vertical-align:center"><img alt="Creative Commons License" src="https://i.creativecommons.org/l/by/4.0/88x31.png"/></td>
  </tr>
</tbody>
</table>
